# Extracting the comments corresponding to the selected posts

In [ ]:
import datetime 
import pandas as pd
import zstandard
import os
import json
from datetime import datetime
import logging.handlers
import datetime as dt

In [ ]:
filtered_df = pd.read_csv(r'./filtered_dfs/filtered_wallstreetbets2017_2018_submissions.csv')
filtered_df.head(20)

,post_id,username,date,timestamp,title,text,score,comment_num,contains_keyword
0,5llrbf,donstone666,2017-01-02,15:32:10,Bitcoin/USD new all-time high!,NaN,1,1,True
1,5m3o1i,soconnoriv,2017-01-05,02:30:14,"(Bitcoin) Well I'll be dammed, this guy called...",NaN,1,1,True
2,5m6qy0,ITradeBaconFutures,2017-01-05,14:45:52,Bitcoin,There's a reason we keep that shit off of this...,16,58,True
3,5muk1v,nachokitty,2017-01-09,01:01:32,Why are you not buying bitcoin when it's so lo...,NaN,1,1,True
4,5n7vrz,BobsBurgers3Bitcoin,2017-01-10,21:38:11,A bitcoin ETF would have 'very significant ups...,NaN,1,2,True
5,5o797v,_nur,2017-01-15,23:33:30,How $XRP plans to take on Bitcoin,NaN,1,1,True
6,5p4dld,volatilebic,2017-01-20,15:08:16,Bitcoin ETF,NaN,5,0,True
7,5v4345,WildFireca,2017-02-20,11:38:20,You can bet on if the Bitcoin ETF will get app...,NaN,55,43,True
8,5vy36v,sweeptheleg55,2017-02-24,15:34:28,Bitcoin ETF play,Just an FYI on a Yolo play. \n\nThere's an etf...,2,22,True
9,5x5d5y,Senor_Suenos,2017-03-02,19:31:07,Bitcoin is now worth more than an ounce of gol...,NaN,2,0,True


In [102]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2092 entries, 0 to 2091
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   post_id           2092 non-null   object
 1   username          2092 non-null   object
 2   date              2092 non-null   object
 3   timestamp         2092 non-null   object
 4   title             2092 non-null   object
 5   text              964 non-null    object
 6   score             2092 non-null   int64 
 7   comment_num       2092 non-null   int64 
 8   contains_keyword  2092 non-null   bool  
dtypes: bool(1), int64(2), object(6)
memory usage: 132.9+ KB


In [3]:
filepath = r'./reddit/subreddits24/wallstreetbets_comments.zst'

In [85]:
filtered_dfs = os.listdir('./filtered_dfs/')
filtered_dfs.sort()
filtered_dfs = [i for i in filtered_dfs if 'filtered_wallstreetbets' in i]
filtered_dfs.pop(-1)
filtered_dfs.pop(-1)
filtered_dfs

['filtered_wallstreetbets2017_2018_submissions.csv',
 'filtered_wallstreetbets2019_2020_submissions.csv',
 'filtered_wallstreetbets2021_submissions.csv']

The code again was manually changed and runned for each DataFrame for memory issue reasons.

In [6]:
# got this from GitHub https://github.com/Watchful1/PushshiftDumps/tree/master


log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		log.info(f"Decoding error with {bytes_read:,} bytes, reading another chunk")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)

			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]

		reader.close()


data_list = []
post_id_list = []

#file_path = sys.argv[1]
file_size = os.stat(filepath).st_size
file_lines = 0
file_bytes_processed = 0
created = None
field = "subreddit"
value = "wallstreetbets"
bad_lines = 0

target_date_start = datetime(2017, 1, 1)
target_date_end = datetime(2019, 1, 1)

# try:
for line, file_bytes_processed in read_lines_zst(filepath):
    try:
        #print('1')
        obj = json.loads(line)
        created = datetime.utcfromtimestamp(int(obj['created_utc']))
        post_id = obj['permalink'].split('/')[4]
        
        #print(created)
        if post_id in filtered_df['post_id'].values and created <= target_date_end and created >= target_date_start:
            if field in obj and obj[field] == value:
                data_list.append(obj)
                post_id_list.append(post_id)
                
        '''if post_id in filtered_df_wallstreetbets2019_2020['post_id'].values and created <= target_date_end_wallstreetbets2019_2020 and created >= target_date_start_wallstreetbets2019_2020:
            if field in obj and obj[field] == value:
                data_list_wallstreetbets2019_2020.append(obj)
                post_id_list_wallstreetbets2019_2020.append(post_id)
                
        if post_id in filtered_df_wallstreetbets2021['post_id'].values and created <= target_date_end_wallstreetbets2021 and created >= target_date_start_wallstreetbets2021:
            if field in obj and obj[field] == value:
                data_list_wallstreetbets2021.append(obj)
                post_id_list_wallstreetbets2021.append(post_id)'''
        
                #print('done')
        #temp = obj[field] == value
    except (KeyError, json.JSONDecodeError) as err:
        bad_lines += 1
    file_lines += 1
    if file_lines % 100000 == 0:
        log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {file_bytes_processed:,}:{(file_bytes_processed / file_size) * 100:.0f}%")

# except Exception as err:
# 	log.info(err)

log.info(f"Complete : {file_lines:,} : {bad_lines:,}")

print('num post extracted:', len(data_list))

/var/folders/qc/7382bn3j7vq46cnb3_4lsqk40000gn/T/ipykernel_9228/1270848615.py:62: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  created = datetime.utcfromtimestamp(int(obj['created_utc']))
2015-08-17 00:53:51 : 100,000 : 100,000 : 29,491,875:0%
2015-08-17 00:53:51 : 100,000 : 100,000 : 29,491,875:0%
2015-12-28 20:01:26 : 200,000 : 200,000 : 29,491,875:0%
2015-12-28 20:01:26 : 200,000 : 200,000 : 29,491,875:0%
2016-04-04 18:18:56 : 300,000 : 300,000 : 57,017,625:1%
2016-04-04 18:18:56 : 300,000 : 300,000 : 57,017,625:1%
2016-06-15 23:14:02 : 400,000 : 400,000 : 57,017,625:1%
2016-06-15 23:14:02 : 400,000 : 400,000 : 57,017,625:1%
2016-08-10 21:02:19 : 500,000 : 500,000 : 57,017,625:1%
2016-08-10 21:02:19 : 500,000 : 500,000 : 57,017,625:1%
2016-10-05 02:56:21 : 600,000 : 600,000 : 83,232,625:1%
2016-

KeyboardInterrupt: 

In [7]:
data_list[4]

{'author': 'NotMyMcChicken',
 'author_flair_css_class': None,
 'author_flair_text': None,
 'body': '5.6 now. This sub is fucking retarded as usual. ',
 'can_gild': True,
 'controversiality': 0,
 'created_utc': 1508415719,
 'distinguished': None,
 'edited': False,
 'gilded': 0,
 'id': 'dol1lsj',
 'is_submitter': False,
 'link_id': 't3_6dhwyk',
 'parent_id': 't1_dlpyz50',
 'permalink': '/r/wallstreetbets/comments/6dhwyk/psa_bitcoin_is_at_an_alltime_high_of_2400_per_coin/dol1lsj/',
 'retrieved_on': 1509977294,
 'score': 3,
 'stickied': False,
 'subreddit': 'wallstreetbets',
 'subreddit_id': 't5_2th52'}

In [8]:
len(data_list)

490

In [9]:
filtered_df['comment_num'].sum()

2720

In [10]:
df_comments = pd.DataFrame()

df_comments['post_id'] = post_id_list
df_comments['comment_id'] = [comment['id'] for comment in data_list]
df_comments['username'] = [comment['author'] for comment in data_list]
df_comments['date'] = [datetime.utcfromtimestamp(int(comment['created_utc'])).date() for comment in data_list]
df_comments['timestamp'] = [datetime.utcfromtimestamp(int(comment['created_utc'])).time() for comment in data_list]
df_comments['text'] = [comment['body'] for comment in data_list]
df_comments['score'] = [comment['score'] for comment in data_list]

df_comments.head(10)

/var/folders/qc/7382bn3j7vq46cnb3_4lsqk40000gn/T/ipykernel_9228/4141128625.py:6: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  df_comments['date'] = [datetime.utcfromtimestamp(int(comment['created_utc'])).date() for comment in data_list]
/var/folders/qc/7382bn3j7vq46cnb3_4lsqk40000gn/T/ipykernel_9228/4141128625.py:7: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  df_comments['timestamp'] = [datetime.utcfromtimestamp(int(comment['created_utc'])).time() for comment in data_list]


,post_id,comment_id,username,date,timestamp,text,score
0,6diqx8,dns7juz,theMightyJoosh,2017-10-02,01:59:16,FUCK NO YOLO IT TO $1M,1
1,70b7n2,dns7xa6,theMightyJoosh,2017-10-02,02:07:21,ARK below $3 now and OMG just below $10. Thoug...,1
2,70b7n2,dns8yrx,juggernaut_alpha,2017-10-02,02:31:30,I think both are great hold. I’m holding more ...,1
3,76zapq,dohr9r7,im_not_a_grill,2017-10-17,15:31:59,"Forget penny stocks, this shit will get pumped...",1
4,6dhwyk,dol1lsj,NotMyMcChicken,2017-10-19,12:21:59,5.6 now. This sub is fucking retarded as usual.,3
5,6f90nm,dolv8oa,[deleted],2017-10-19,21:21:45,[removed],1
6,6diqx8,dolwjk3,[deleted],2017-10-19,21:46:12,[removed],1
7,6dhwyk,dowf21t,ArrayBoy,2017-10-26,03:06:54,Still waiting,1
8,6dhwyk,doy8pfd,Jabroni421,2017-10-27,03:35:05,"6,000 right now lolz",1
9,79xcl1,dp5gh24,[deleted],2017-10-31,17:32:33,[removed],1


In [ ]:
if os.path.exists('./comments'):
    df_comments.to_csv('./comments/wallstreetbets2024_comments.csv', index=False)

else:
    os.makedirs('./comments')
    df_comments.to_csv('./comments/wallstreetbets2024_comments.csv', index=False)